In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow import keras
from keras import layers
from keras import losses

print(tf.__version__)

2.14.0


## Построение модели
На вход идут данные по коррелирующей дисциплине:
1. Доля посешенных лекций в первом промежутке ($\frac {посещено} {общее\_количество}$)
2. Доля посешенных лекций в первом промежутке ($\frac {посещено} {общее\_количество}$)
3. Доля посешенных лекций в третьем промежутке ($\frac {посещено} {общее\_количество}$)
4. Доля посешенных практик в первом промежутке ($\frac {посещено} {общее\_количество}$)
5. Доля посешенных практик в первом промежутке ($\frac {посещено} {общее\_количество}$)
6. Доля посешенных практик в третьем промежутке ($\frac {посещено} {общее\_количество}$)
7. Оценка за КТ 1 ($\frac {оценка} {5}$)
8. Оценка за КТ 2 ($\frac {оценка} {5}$)
9. Оценка за экзамен \[0,5\] если зачет \[0;5\] ($\frac {оценка} {5}$)
10. Средний балл по контрольным ($\frac {балл} {5}$)
11. Доля выполненных лабораторных работ ($\frac {выполнено} {общее\_количество}$)
12. Уровень корреляции дисциплины с прогнозируемой



Также идут следующие метки:
- Реальная оценка по дисциплине

На выходе:
- Уровни уверенности в прогнозируемой оценке

In [2]:
model = keras.Sequential([
    layers.Dense(5, input_dim=12, activation='softmax'),
    layers.Dense(1, activation='softmax')
])

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

## Подготовка данных

In [3]:
table = pd.read_csv("C:\\MyFolders\\ASU2304\\TEST\\data\\final\\data.csv").drop('Unnamed: 0', axis=1)

table.head(5)

,connection_id,semester,discipline_name,lecture_1,lecture_2,lecture_3,practice_1,practice_2,practice_3,kt_1,kt_2,exam,controls,labs,encoding_com
0,157087,0,Информатика,0.666667,0.500000,1.000000,0.777778,0.818182,0.666667,2.0,2.0,2.0,1,1,27648.0
1,157087,0,Компьютерная графика,0.666667,0.500000,0.500000,0.500000,1.000000,1.000000,1.0,2.0,4.0,1,1,16384.0
2,157087,0,Математика,1.000000,0.714286,0.750000,0.500000,0.857143,0.750000,0.0,0.0,1.0,1,1,16448.0
3,157087,0,Программирование,0.857143,0.166667,0.333333,0.333333,0.333333,0.000000,3.0,1.0,4.0,1,1,3072.0
4,157087,1,История,0.333333,0.500000,0.000000,0.500000,0.666667,0.142857,2.0,1.0,2.0,1,1,2.0


In [4]:
# Разбиваем датафрейм на каждого студента
students = table.groupby('connection_id')

student_tables = [students.get_group(x) for x in students.groups]

for i in range (len(student_tables)):
    student_tables[i] = student_tables[i].reset_index().drop('index', axis=1)
    
student_tables[2]

,connection_id,semester,discipline_name,lecture_1,lecture_2,lecture_3,practice_1,practice_2,practice_3,kt_1,kt_2,exam,controls,labs,encoding_com
0,157091,0,Введение в информатику и вычислительную технику,0.833333,1.000000,0.750000,0.000000,0.000000,0.000000,2.0,1.5,2.0,1,1,4194304.0
1,157091,0,Дискретная математика,0.857143,0.750000,0.500000,0.857143,1.000000,0.666667,2.0,3.0,3.0,1,1,16448.0
2,157091,0,Информатика,1.000000,1.000000,0.500000,0.000000,0.000000,0.000000,0.0,2.0,1.0,1,1,27648.0
3,157091,0,История,1.000000,1.000000,1.000000,0.666667,1.000000,0.666667,3.0,3.0,2.0,1,1,2.0
4,157091,0,Математика,0.857143,0.857143,0.500000,1.000000,1.000000,0.666667,4.0,3.0,3.0,1,1,16448.0
5,157091,0,Программирование,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.0,1.5,2.0,1,1,3072.0
6,157091,0,Русский язык и культура речи,0.666667,1.000000,0.000000,1.000000,0.500000,0.500000,4.0,4.0,2.0,1,1,16.0
7,157091,1,Математика,0.800000,0.750000,0.833333,1.000000,0.833333,0.818182,2.0,2.0,2.0,1,1,16448.0
8,157091,1,Математическая логика и теория алгоритмов,1.000000,0.750000,0.833333,1.000000,1.000000,1.000000,3.0,2.0,3.0,1,1,64.0
9,157091,1,Основы информационных технологий,1.000000,0.800000,0.666667,0.000000,0.000000,0.000000,2.0,2.0,2.0,1,1,4196352.0


In [5]:
# 2 таблицы. 1 - для обучения 2 - для проверки

table_train = []
table_test = []


test_count = len(student_tables)//10
for i in range(test_count):
    table_test.append(student_tables[-i])
    
for i in range(len(student_tables) - test_count):
    table_train.append(student_tables[i])

In [6]:
# Фреймы данных и меток
train_data = pd.DataFrame()
train_label = pd.DataFrame()

In [7]:
# Начинаем формировать датасет
duplications = []

for i in table_train:
    grps = i.groupby('semester')
    discp_count = grps['discipline_name'].count().reset_index().rename(columns={'discipline_name':'count'})
    discp_count = discp_count.to_numpy()
    
    my_duplications = []
    for j in range(len(grps.groups)):
        for z in range(discp_count[j, 1]):
            to_concat = []
            to_concat.append(train_data)
            to_concat += [grps.get_group(x) for x in range(j)]
            train_data = pd.concat(to_concat, ignore_index=True)
        my_duplications.append(sum(len(grps.get_group(x)) for x in range(j)))
        
    duplications.append(my_duplications)

train_data = train_data[['lecture_1', 'lecture_2', 'lecture_3', 'practice_1', 'practice_2', 'practice_3', 'kt_1', 'kt_2', 'exam', 'controls', 'labs', 'encoding_com']]
train_data

,lecture_1,lecture_2,lecture_3,practice_1,practice_2,practice_3,kt_1,kt_2,exam,controls,labs,encoding_com
0,0.666667,0.500000,1.000000,0.777778,0.818182,0.666667,2.0,2.0,2.0,1,1,27648.0
1,0.666667,0.500000,0.500000,0.500000,1.000000,1.000000,1.0,2.0,4.0,1,1,16384.0
2,1.000000,0.714286,0.750000,0.500000,0.857143,0.750000,0.0,0.0,1.0,1,1,16448.0
3,0.857143,0.166667,0.333333,0.333333,0.333333,0.000000,3.0,1.0,4.0,1,1,3072.0
4,0.666667,0.500000,1.000000,0.777778,0.818182,0.666667,2.0,2.0,2.0,1,1,27648.0
...,...,...,...,...,...,...,...,...,...,...,...,...
30922,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,1,1,2099200.0
30923,0.000000,0.000000,0.000000,0.000000,0.136364,0.272727,0.0,0.0,0.0,1,1,4194368.0
30924,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,1,1,4194308.0
30925,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,1,1,128.0


In [8]:
# Начинаем формирование меток
counter = 0

def set_dup_cout(row):
    if row['semester'] != 0:
        row['repeat'] = int(duplications[counter][row['semester']])
        
    return row

test = 0
for i in table_train:
    i['repeat'] = -1
    i = i.apply(set_dup_cout, axis=1).dropna()
    i = i.loc[i['repeat'] != -1]
    i = i.loc[i.index.repeat(i['repeat'])].reset_index(drop=True)
    
    to_concat = []
    to_concat.append(train_label)
    to_concat.append(i)
    train_label = pd.concat(to_concat, ignore_index=True)
    
    counter += 1
    
train_label = train_label.drop('repeat', axis=1)
train_label = train_label[['kt_1', 'kt_2', 'exam', 'encoding_com']].rename(columns={'encoding_com':'com'})
train_label

,kt_1,kt_2,exam,com
0,2.0,1.0,2.0,2.0
1,2.0,1.0,2.0,2.0
2,2.0,1.0,2.0,2.0
3,2.0,1.0,2.0,2.0
4,0.0,0.0,1.0,16448.0
...,...,...,...,...
30922,0.0,0.0,0.0,2099200.0
30923,0.0,0.0,0.0,2099200.0
30924,0.0,0.0,0.0,2099200.0
30925,0.0,0.0,0.0,2099200.0


In [9]:
# Завершаем разметку
def averageMy(row):
    row['mark'] = (row['kt_1'] + row['kt_2'] + row['exam'])/3
    if row['mark'] == 5:
        row['mark'] == 4
    elif row['mark'] == 4:
        row['mark'] == 3
    elif row['mark'] == 3:
        row['mark'] == 2
    elif row['mark'] == 2:
        row['mark'] == 1
    return row

train_label = train_label.apply(averageMy, axis=1)
train_label = train_label[['mark', 'com']]
train_label

KeyboardInterrupt: 

In [ ]:
train_data = pd.concat([train_data, train_label['com']], axis=1)
train_label.drop('com', axis=1, inplace=True)
train_data

,lecture_1,lecture_2,lecture_3,practice_1,practice_2,practice_3,kt_1,kt_2,exam,controls,labs,encoding_com,com
0,0.666667,0.500000,1.000000,0.777778,0.818182,0.666667,2.0,2.0,2.0,1,1,27648.0,2.0
1,0.666667,0.500000,0.500000,0.500000,1.000000,1.000000,1.0,2.0,4.0,1,1,16384.0,2.0
2,1.000000,0.714286,0.750000,0.500000,0.857143,0.750000,0.0,0.0,1.0,1,1,16448.0,2.0
3,0.857143,0.166667,0.333333,0.333333,0.333333,0.000000,3.0,1.0,4.0,1,1,3072.0,2.0
4,0.666667,0.500000,1.000000,0.777778,0.818182,0.666667,2.0,2.0,2.0,1,1,27648.0,16448.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30922,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,1,1,2099200.0,2099200.0
30923,0.000000,0.000000,0.000000,0.000000,0.136364,0.272727,0.0,0.0,0.0,1,1,4194368.0,2099200.0
30924,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,1,1,4194308.0,2099200.0
30925,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,1,1,128.0,2099200.0


In [ ]:
# Завершаем датасет
def cor(row):
    one = row['encoding_com']
    two = row['com']
    
    one_flags = []
    two_flags = []
    
    for i in range(30):
        localFlag = False
        if(pow(2, 29-i) <= one):
            one_flags.append(True)
            one -= pow(2, 29-i)
        else:
            one_flags.append(False)
            
        if(pow(2, 29-i) <= two):
            two_flags.append(True)
            two -= pow(2, 29-i)
        else:
            two_flags.append(False)
            
    for i in range(30):
        if one_flags[i] and two_flags[i]:
            row['cor'] = 0.8
            return row
        
    one_larg_flags = [False, False, False]
    two_larg_flags = [False, False, False]
    
    if one//1048576 >= 1:
        one_larg_flags[0] = True
    elif one//1024 >= 1:
        one_larg_flags[1] = True
    else:
        one_larg_flags[2] = True
        
    if two//1048576 >= 1:
        two_larg_flags[0] = True
    elif two//1024 >= 1:
        two_larg_flags[1] = True
    else:
        two_larg_flags[2] = True
    
    for i in range(3):
        if one_larg_flags[i] and two_larg_flags[i]:
            row['cor'] = 0.5
            return row
    
    row['cor'] = 0.2
    
    return row

train_data = train_data.apply(cor, axis=1).drop(['com', 'encoding_com'], axis=1)

def mark_norm(row):
    row['kt_1'] = row['kt_1']/4
    row['kt_2'] = row['kt_2']/4
    row['exam'] = row['exam']/4
    
    return row

train_data = train_data.apply(mark_norm, axis=1)

train_data

,lecture_1,lecture_2,lecture_3,practice_1,practice_2,practice_3,kt_1,kt_2,exam,controls,labs,cor
0,0.666667,0.500000,1.000000,0.777778,0.818182,0.666667,2.0,2.0,2.0,1.0,1.0,0.5
1,0.666667,0.500000,0.500000,0.500000,1.000000,1.000000,1.0,2.0,4.0,1.0,1.0,0.5
2,1.000000,0.714286,0.750000,0.500000,0.857143,0.750000,0.0,0.0,1.0,1.0,1.0,0.5
3,0.857143,0.166667,0.333333,0.333333,0.333333,0.000000,3.0,1.0,4.0,1.0,1.0,0.5
4,0.666667,0.500000,1.000000,0.777778,0.818182,0.666667,2.0,2.0,2.0,1.0,1.0,0.8
...,...,...,...,...,...,...,...,...,...,...,...,...
30922,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.0,1.0,0.8
30923,0.000000,0.000000,0.000000,0.000000,0.136364,0.272727,0.0,0.0,0.0,1.0,1.0,0.5
30924,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.0,1.0,0.5
30925,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.0,1.0,0.5


Обучение

In [ ]:
model = keras.Sequential([
    layers.Dense(6, input_dim=12, activation='relu'),
    layers.Dense(5)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(train_data, train_label, epochs=5, batch_size=10)

Epoch 1/5
3093/3093 [==============================] - 3s 948us/step - loss: 1.4586 - accuracy: 0.0624
Epoch 2/5
3093/3093 [==============================] - 3s 1ms/step - loss: 1.3558 - accuracy: 0.0701
Epoch 3/5
3093/3093 [==============================] - 3s 1ms/step - loss: 1.3471 - accuracy: 0.0710
Epoch 4/5
3093/3093 [==============================] - 3s 965us/step - loss: 1.3417 - accuracy: 0.0721
Epoch 5/5
1353/3093 [============>.................] - ETA: 1s - loss: 1.3395 - accuracy: 0.0713

# Следующий уровень
На вход идет:
1. Средняя предсказанная оценка
2. Средний уровень корреляции
3. Оценка по предсказанию максимальной кореляции
4. Максимальная корреляция
5. Оценка по предсказанию минимальной кореляции
6. Минимальная корреляция

Метки:
- Реальный полученный балл

На выход:
- Итоговое пердсказание

In [ ]:
model1 = keras.Sequential([
    layers.Dense(6, input_dim=6, activation='elu'),
    layers.Dense(5, activation='sigmoid'),
    layers.Dense(1, activation='softmax')
])

model1.compile(optimizer='adam', loss = losses.SparseCategoricalCrossentropy(from_logits=True), metrics = ['accuracy'])

## ЗАМЕТКИ
1. В сборке данных есть ошибка - дублируются дисциплины. Это связано с тем, что идет объеденение таблиц компетенций для разных направлений. Исправлять на текущий момент, потому что это надо переделывать таблицы.
<br>Фикс: Добавить в таблицы Володи направление для каждой строки, затем соеденить по направлению с основным датафреймом.